In [3]:
import json
import pandas as pd
import random
import nltk
from nltk.corpus import sentiwordnet as swn
from nltk.stem import *
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer

In [4]:
def removeStopWords(text):
    #stopwords = nltk.corpus.stopwords.words('english')
    stopwords = ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', 'her', 'hers', 'herself', 'it', 'its', 'itself', 'they'
                 , 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that'
                 , 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 
                 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and','to']
    #print(stopwords)
    #stopwords.remove('not')
    content = [w for w in text if w.lower() not in stopwords]
    return content

In [9]:
text = nltk.word_tokenize("They did not never to permit us to obtain the refuse permit")
filteredText = removeStopWords(text)
filteredText

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', 'her', 'hers', 'herself', 'it', 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'to']


['not', 'never', 'permit', 'us', 'obtain', 'refuse', 'permit']

In [5]:
with open("/u/ggabra/Search Project/yelp_academic_dataset_review.json") as f:
    data = pd.DataFrame(json.loads(line) for line in f)

In [6]:
rows = random.sample(range(0,len(data)-1),268500)
testData = data.iloc[rows,[3,4]].copy() 
testData=testData.reset_index(drop=True)
trainData = data.drop(rows)
trainData=trainData.reset_index(drop=True)
trainData.drop(trainData.columns[[0,1,2,5,6,7]], axis=1, inplace=True)

In [7]:
sampleData = trainData[:10000]
sampleData.head(1)

,stars,text
0,4,"Mr Hoagie is an institution. Walking in, it do..."


In [9]:
bigList=[]
train_label=[]
JJRB = set(['JJ', 'RB','JJR'])
for index,row in sampleData.iterrows():
    #lst = []
    wordList = []
    tokens = removeStopWords(nltk.word_tokenize(row[1]))
    tags = nltk.pos_tag(tokens)
    #print(tags)
    #for tag in tags:
    #    if tag[1] == 'JJ' or tag[1] == 'RB': 
    #        wordList.append(tag[0])
    #wordString = ' '.join([ tag[0] for tag in tags if tag[1] in JJRB ]).lower()        
    #bigList.append(wordString)
    bigList.append(' '.join([ tag[0] for tag in tags if tag[1] in JJRB ]).lower())
    train_label.append(row[0])    

In [10]:
bigList[50],train_label[1]

('nice enough enough good enough absolutely wrong not full great back not wow great just good enough',
 5)

In [17]:
sampleTestData = testData[:1000]

testList=[]
test_label=[]
JJRB = set(['JJ', 'RB',' JJR'])
for index,row in sampleTestData.iterrows():
    #lst = []
    wordList = []
    tokens = removeStopWords(nltk.word_tokenize(row[1]))
    tags = nltk.pos_tag(tokens)
    #print(tags)
    #for tag in tags:
    #    if tag[1] == 'JJ' or tag[1] == 'RB': 
    #        wordList.append(tag[0])
    #wordString = ' '.join([ tag[0] for tag in tags if tag[1] in JJRB ]).lower()        
    #bigList.append(wordString)
    testList.append(' '.join([ tag[0] for tag in tags if tag[1] in JJRB ]).lower())
    test_label.append(row[0]) 

In [18]:
import nltk
text = nltk.word_tokenize("the drive-thru")
tagWordList = nltk.pos_tag(text)
tagWordList

[('the', 'DT'), ('drive-thru', 'NN')]

In [28]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import SGDClassifier, LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
import numpy
text_clf = Pipeline([('vect', CountVectorizer(ngram_range=[1,2])),
                      ('tfidf', TfidfTransformer()),
                      ('clf', SGDClassifier(loss='hinge', penalty='l2',
                                            alpha=1e-3, n_iter=5, random_state=42))     #change classifier names here
 ])
_ = text_clf.fit(bigList, train_label)

In [29]:
predicted_label = text_clf.predict(testList)
numpy.mean(predicted_label == test_label)

0.54400000000000004

In [30]:
from sklearn.metrics import mean_squared_error
from math import sqrt
sqrt(mean_squared_error(test_label, predicted_label))

1.2099586769803339

In [35]:
from sklearn import metrics
#print(metrics.classification_report(twenty_test.target, predicted, target_names=twenty_test.target_names))
metrics.confusion_matrix(test_label, predicted_label)

array([[ 403,   13,   40,   25,  125],
       [ 152,   38,   63,   51,   85],
       [  85,   26,  127,  182,  161],
       [  47,   12,   92,  356,  815],
       [  63,    7,   40,  187, 1805]])

In [41]:
targetNames = []
targetNames.append('1')
targetNames.append('2')
targetNames.append('3')
targetNames.append('4')
targetNames.append('5')
print(metrics.classification_report(test_label, predicted_label,
   target_names=targetNames))

             precision    recall  f1-score   support

          1       0.54      0.67      0.59       606
          2       0.40      0.10      0.16       389
          3       0.35      0.22      0.27       581
          4       0.44      0.27      0.34      1322
          5       0.60      0.86      0.71      2102

avg / total       0.51      0.55      0.50      5000



In [ ]:
print('hi')